In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import random
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
#0=incorrect 1 correct 0-wallsit 1-plank
#loading the dataset into array format 
data=pd.read_csv("squat.csv",header=1)
# Shuffle the rows of the DataFrame
shuffled = data.sample(frac=1).reset_index(drop=True)
#print(shuffled.head())
shuffled.shape

(466, 18)

In [7]:
from sklearn.model_selection import train_test_split
# Assuming the last column is the target variable (y) and the rest are features (X)
X = data.iloc[:, :-1]  # All columns except the last one
y = data.iloc[:, -1]   # Last column

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)


(372, 17)


In [8]:
#creating the neural network model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import Adam

# Build the neural network
model = Sequential()
# Input layer (5 inputs)
model.add(Input(shape=(X_train.shape[1],)))

model.add(Dense(10, activation='relu')) 
# Additional hidden layer (optional)

model.add(Dense(5, activation='relu'))  # 5 neurons in the hidden layer

# Output layer (binary classification, so 1 neuron with sigmoid activation)
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           180 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241 (964.00 B)

 Trainable params: 241 (964.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
#compiling the model(configuring the model)
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [10]:
#training the model
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Create the callbacks
callback_a = ModelCheckpoint(filepath="my_best_model.keras", monitor='val_loss', save_best_only=True)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)



In [11]:
history= model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=256,batch_size=20,callbacks=[callback_a,callback_b],verbose=1)

Epoch 1/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6569 - loss: 2.2480 - val_accuracy: 0.8298 - val_loss: 0.4243
Epoch 2/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8548 - loss: 0.4270 - val_accuracy: 0.8298 - val_loss: 0.3282
Epoch 3/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8539 - loss: 0.3408 - val_accuracy: 0.9362 - val_loss: 0.2503
Epoch 4/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8856 - loss: 0.3155 - val_accuracy: 0.9362 - val_loss: 0.2231
Epoch 5/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9135 - loss: 0.2855 - val_accuracy: 0.8511 - val_loss: 0.3148
Epoch 6/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8688 - loss: 0.3378 - val_accuracy: 0.8617 - val_loss: 0.2811
Epoch 7/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8960 - loss: 0.2785 - val_accuracy: 0.8723 - val_loss: 0.2877
Epoch 8/256
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8474 - loss: 0.3284 - val_accuracy: 0.8723 - 

In [12]:
#checking the learning curves
print(history.params)


{'verbose': 1, 'epochs': 256, 'steps': 19}


In [13]:
#evaluate the model
scores=model.evaluate(X_train,y_train)
print(model.metrics_names)
print(scores)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9866 - loss: 0.0688 
['loss', 'compile_metrics']
[0.06466872990131378, 0.9865591526031494]

compile_metrics: 98.66%


In [14]:
#evaluating on validating set
scores=model.evaluate(X_test,y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9908 - loss: 0.0249 

compile_metrics: 98.94%


In [15]:
print(X_test[0:5])
print(y_test[0:5])

     2  0.0  0.0.1  0.0.2  0.0.3  0.0.4  0.0.5       0.0.6       0.0.7  \
55   2  0.0    0.0    0.0    0.0    0.0    0.0    0.000000    0.000000   
63   2  0.0    0.0    0.0    0.0    0.0    0.0   96.591150   96.591150   
33   2  0.0    0.0    0.0    0.0    0.0    0.0    0.000000    0.000000   
461  2  0.0    0.0    0.0    0.0    0.0    0.0  139.732184  139.732184   
72   2  0.0    0.0    0.0    0.0    0.0    0.0   90.415267   90.415267   

          0.0.8       0.0.9  0.0.10  0.0.11  0.0.12  0.0.13  0.0.14  0.0.15  
55    79.878591    0.000000     0.0     0.0     0.0     0.0     0.0     0.0  
63    83.478251   83.577068     0.0     0.0     0.0     0.0     0.0     0.0  
33   176.724037  163.864628     0.0     0.0     0.0     0.0     0.0     0.0  
461  156.939403  158.882775     0.0     0.0     0.0     0.0     0.0     0.0  
72    82.002406   86.428903     0.0     0.0     0.0     0.0     0.0     0.0  
55     1.0
63     1.0
33     0.0
461    0.0
72     1.0
Name: 0.0.16, dtype: float64


In [16]:
prediction=model.predict(X_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [12]:
print(prediction[0:10])

[[9.7242850e-01]
 [9.9828631e-01]
 [3.9375612e-19]
 [1.6502441e-03]
 [9.9988782e-01]
 [4.0570080e-07]
 [0.0000000e+00]
 [1.0664665e-02]
 [9.9995571e-01]
 [7.3840944e-08]]


In [13]:
print(prediction[0:10].round())

[[1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]]
